N.b. This requries a local large language model to be running. Its rquired that users download OLLAMA before they can do anything https://github.com/ollama/ollama 

Open WebUI and Ollama server need to be running for this code to work

I.e. #ollama run llama3.2

In [20]:
import requests
import json


# 🔹 Replace with your actual API key
API_KEY = "###"
WEBUI_URL = "http://localhost:3000/api/chat/completions"

In [21]:

publications = []

with open ('./app/data/publications.json') as f:
    publications = json.load(f)

In [22]:
paper_abstracts = [x['abstract'] for x in publications]
paper_doi = [x['paper_doi'] for x in publications]
paper_abstracts[1]
abstract = paper_abstracts[2005]

print(abstract)

<h4>Background</h4>The omicron (B.1.1.529) variant of SARS-CoV-2 is highly transmissible and escapes vaccine-induced immunity. We aimed to describe outcomes due to COVID-19 during the omicron outbreak compared with the prevaccination period and alpha (B.1.1.7) and delta (B.1.617.2) waves in patients with cancer in Europe.<h4>Methods</h4>In this retrospective analysis of the multicentre OnCovid Registry study, we recruited patients aged 18 years or older with laboratory-confirmed diagnosis of SARS-CoV-2, who had a history of solid or haematological malignancy that was either active or in remission. Patient were recruited from 37 oncology centres from UK, Italy, Spain, France, Belgium, and Germany. Participants were followed up from COVID-19 diagnosis until death or loss to follow-up, while being treated as per standard of care. For this analysis, we excluded data from centres that did not actively enter new data after March 1, 2021 (in France, Germany, and Belgium). We compared measures

In [ ]:
import requests
import re

def can_you_find_a_dataset_webui(description, model):
    contents=f"Can you find a name of any dataset(s) in the following description from a paper abstract, give me a comma separated list of names of possible dataset names, if you dont think there is one, give me no response - not a single word:\n\n {description}"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": model,
        "messages": [{"role": "user", "content": contents}]
    }

    response = requests.post(WEBUI_URL, headers=headers, json=payload, stream=True)

    is_google_genai = "google_genai" in model.lower()
    full_content = []

    try:
        if is_google_genai:
            response_text = ""
            for line in response.iter_lines():
                if not line.strip():
                    continue  # Ignore empty lines
                
                line = line.decode("utf-8")  # Convert from bytes to string
                response_text += line + " "  # Append to full text

            # Extract all content values manually using regex
            matches = re.findall(r'"content":\s?"(.*?)"', response_text)
            full_content = [match.strip() for match in matches if match.strip()]

            # Join extracted content into a single string
            content = "".join(full_content).replace("\\n", "")
            content = [x.lstrip().rstrip() for x in content.split(",")]
        else:
            content = [x.lstrip().rstrip() for x in response.text.split(",")]

        return content if content else []

    except requests.exceptions.RequestException as e:
        return []

# Test with a sample abstract
abstract = "This study uses the UK Biobank and Genomics England dataset."
dataset_names = can_you_find_a_dataset_webui(abstract, "google_genai.gemini-2.0-flash")
print(dataset_names)


['UKBiobank', 'Genomics England']
['UKBiobank', 'Genomics England']


In [27]:
# 🔹 List available models to check correct name
def list_models():
    headers = {"Authorization": f"Bearer {API_KEY}"}
    response = requests.get("http://localhost:3000/api/models", headers=headers)
    
    try:
        available_models = response.json()
        # Extract model names
        model_names = [model["id"] for model in available_models["data"] if "name" in model]

        return model_names
    except requests.exceptions.RequestException as e:
        return f"Error fetching models: {e}"

# 🔹 Get available models and use a correct one
# Extract model names
model_names = list_models()
# Print the extracted list
print(model_names)

['google_genai.gemini-1.0-pro-latest', 'google_genai.gemini-1.0-pro', 'google_genai.gemini-pro', 'google_genai.gemini-1.0-pro-001', 'google_genai.gemini-1.0-pro-vision-latest', 'google_genai.gemini-pro-vision', 'google_genai.gemini-1.5-pro-latest', 'google_genai.gemini-1.5-pro-001', 'google_genai.gemini-1.5-pro-002', 'google_genai.gemini-1.5-pro', 'google_genai.gemini-1.5-flash-latest', 'google_genai.gemini-1.5-flash-001', 'google_genai.gemini-1.5-flash-001-tuning', 'google_genai.gemini-1.5-flash', 'google_genai.gemini-1.5-flash-002', 'google_genai.gemini-1.5-flash-8b', 'google_genai.gemini-1.5-flash-8b-001', 'google_genai.gemini-1.5-flash-8b-latest', 'google_genai.gemini-1.5-flash-8b-exp-0827', 'google_genai.gemini-1.5-flash-8b-exp-0924', 'google_genai.gemini-2.0-flash-exp', 'google_genai.gemini-2.0-flash', 'google_genai.gemini-2.0-flash-001', 'google_genai.gemini-2.0-flash-lite-preview', 'google_genai.gemini-2.0-flash-lite-preview-02-05', 'google_genai.gemini-2.0-pro-exp', 'google_ge